In [2]:
import numpy as np
import glob
import os
import sys
import scipy

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2
import imutils
import pandas as pd
import re
import datetime

import zipfile

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from sklearn import metrics
from sklearn.utils import shuffle
from skimage import draw
from skimage.io import imread
from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split#import spams

from skimage.util import pad
import skimage

# import time
# import math
# import keras
# from keras.models import *
# from keras.layers import *

# from keras.preprocessing import image
# import keras.backend as K
# import random
# from sklearn.metrics import f1_score
# from skimage.transform import resize
# from skimage.filters import threshold_otsu
# from tqdm import tqdm_notebook as tqdm 
# #from tensorflow.keras.callbacks import TensorBoard

# from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.29
config.gpu_options.allow_growth=True ##to use gpu as needed
config.log_device_placement = True  
sess = tf.Session(config=config)
set_session(sess)
K.tensorflow_backend._get_available_gpus()



 

AttributeError: module 'tensorflow' has no attribute 'RunOptions'

In [ ]:
!nvidia-smi

In [ ]:
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
IMAGE_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input'
MASK_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask'
DISCARD_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard'

MASK_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
IMAGE_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_non_normalized'
MASK_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_non_normalized'
DISCARD_WRITE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard_non_normalized'




In [ ]:


def extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH):
    
    if not os.path.exists(IMAGE_WRITE_PATH):
        os.mkdir(IMAGE_WRITE_PATH)
        print('input directory made')
    else:
        print('input directory exists')

    if not os.path.exists(MASK_WRITE_PATH):
        os.mkdir(MASK_WRITE_PATH)
        print('mask directory made')
    else:
        print('mask directory exists')

    if not os.path.exists(DISCARD_WRITE_PATH):
        os.mkdir(DISCARD_WRITE_PATH)
        print('discard directory made')
    else:
        print('discard directory exists')

    image_list=os.listdir(IMAGE_PATH)

    for image_ID in tqdm(image_list):
        # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
        image_name=os.path.join(IMAGE_PATH,image_ID)
        gray_img=cv2.imread(image_name,0)
        bgr_img=cv2.imread(image_name,1)
        mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
        # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
        mask_name=os.path.join(MASK_PATH,mask_ID)
        mask_img=cv2.imread(mask_name,0)
        ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)



        th,img_otsu=cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        temp_x,temp_y=np.where(img_otsu==255)

        for count_x,x in enumerate(range(img_otsu.shape[0]//512)):
            for count_y,y in enumerate(range(img_otsu.shape[1]//512)):

                temp_otsu=img_otsu[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]
                temp_mask=mask_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512]


                if len(np.where(temp_otsu==0)[0])<0.6*len(np.where(temp_otsu==255)[0]) and len(np.where(temp_mask==255)[0])>=5000:
                    img_write=bgr_img[count_x*512:(count_x+1)*512,count_y*512:(count_y+1)*512,:]

                    write_path_image=os.path.join(IMAGE_WRITE_PATH,image_ID.split('.')[0]+\
                                              "_{}_{}.jpg".format(count_x,count_y))
            # Format of image_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>.jpg
                    cv2.imwrite(write_path_image,img_write)
                    write_path_mask=os.path.join(MASK_WRITE_PATH,mask_ID.split('_mask')[0]+\
                                                  "_{}_{}".format(count_x,count_y)+'_mask.jpg')
            # Format of mask_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>_mask.jpg
                    cv2.imwrite(write_path_mask,temp_mask)
                else:
                    write_path_discard=os.path.join(DISCARD_WRITE_PATH,mask_ID.split('.')[0]+\
                                                  "{}_{}.jpg".format(count_x,count_y))
                    cv2.imwrite(write_path_discard,temp_mask)

#extract_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH)



#extract_patches(MASK_PATH_NN,IMAGE_PATH_NN,IMAGE_WRITE_PATH_NN,MASK_WRITE_PATH_NN,DISCARD_WRITE_PATH_NN)

In [ ]:

# !rm -rf mask_cn_overlap_test
# !rm -rf mask_cn_overlap_train
# !rm -rf mask_cn_overlap_val

# !rm -rf input_cn_overlap_train
# !rm -rf input_cn_overlap_test
# !rm -rf input_cn_overlap_val
# !rm -rf discard_cn_overlap


In [ ]:
def gen_patches(list_images,image_write_path,mask_write_path,discard_path):
    MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
    IMAGE_PATH_NOR='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
    
    for image_ID in tqdm(list_images):
        # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
        image_name=os.path.join(IMAGE_PATH_NOR,image_ID)
        gray_img=cv2.imread(image_name,0)
        bgr_img=cv2.imread(image_name,1)
        mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
        # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
        mask_name=os.path.join(MASK_PATH,mask_ID)
        mask_img=cv2.imread(mask_name,0)
        ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)



        th,img_otsu=cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        x_range=(img_otsu.shape[0]-512)//256 +1
        y_range=(img_otsu.shape[1]-512)//256 +1
    
        for count_x in (range(x_range)):
            start_x_pos=count_x*256
            end_x_pos=start_x_pos+512
            for count_y in (range(y_range)):
                start_y_pos=count_y*256
                end_y_pos=start_y_pos+512
                

                temp_otsu=img_otsu[start_x_pos:end_x_pos,start_y_pos:end_y_pos]
                temp_mask=mask_img[start_x_pos:end_x_pos,start_y_pos:end_y_pos]


                if len(np.where(temp_otsu==0)[0])<0.6*len(np.where(temp_otsu==255)[0]) and len(np.where(temp_mask==255)[0])>=5000:
                    img_write=bgr_img[start_x_pos:end_x_pos,start_y_pos:end_y_pos,:]

                    write_path_image=os.path.join(image_write_path,image_ID.split('.')[0]+\
                                              "_{}_{}.jpg".format(count_x,count_y))
            # Format of image_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>.jpg
                    cv2.imwrite(write_path_image,img_write)
                    write_path_mask=os.path.join(mask_write_path,mask_ID.split('_mask')[0]+\
                                                  "_{}_{}".format(count_x,count_y)+'_mask.jpg')
            # Format of mask_ID now: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_<count_x>_<count_y>_mask.jpg
                    cv2.imwrite(write_path_mask,temp_mask)
                else:
                    write_path_discard=os.path.join(discard_path,mask_ID.split('.')[0]+\
                                                  "{}_{}.jpg".format(count_x,count_y))
                    cv2.imwrite(write_path_discard,temp_mask)
                    



def extract_overlap_patches(MASK_PATH,IMAGE_PATH,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH):
    must_include=['18-00530B_2019-05-07 23_56_22-lv1-11712-16122-7372-7686.jpg',\
        '18-01080B_2019-05-07 21_33_52-lv1-15262-19621-5715-4803.jpg',\
        '18-03912A_2019-05-07 22_55_07-lv1-16713-11566-3121-5791.jpg',\
        '18-04842A_2019-05-07 23_36_39-lv1-21930-4017-5709-5966.jpg',\
        '18-09926A_2019-05-08 00_06_27-lv1-23990-14292-4181-5408.jpg',\
        '18-13347A_2019-05-08 01_31_58-lv1-14057-17758-5693-6244.jpg',\
        '2018_68099_1-1_2019-02-20 23_21_13-lv1-7948-15919-4988-5294.jpg',\
        '2018_72876_1-1_2019-02-21 00_28_52-lv1-31103-26670-3231-3747.jpg',\
        '2018_73834_1-1_2019-02-21 00_16_28-lv1-43278-10364-7406-7672.jpg',\
        '2018_74969_1-1_2019-02-21 00_48_39-lv1-39175-17160-3764-3967.jpg',\
        '2018_83220_1-1_2019-02-20 18_33_11-lv1-58151-37497-5712-3460.jpg',\
        '2019_01246_1-1_2019-02-20 19_23_52-lv1-41847-6127-5995-4660.jpg',\
        '2019_02170_1-2_2019-02-20 19_37_17-lv1-18315-29727-7652-5581.jpg',\
        '2019_03867_1-1_2019-02-20 20_00_32-lv1-36172-15430-6105-5822.jpg',\
        '2019_05944_1-1_2019-02-20 20_42_06-lv1-31069-20357-7018-5934.jpg',\
        '1901940-1_2019-04-30 10_30_47-lv1-39947-11074-3848-5785.jpg',\
        '1800883002_2019-04-30 09_57_31-lv1-28885-30819-3472-3263.jpg',\
        'D201710920_2019-05-21 11_52_57-lv1-33661-31501-7087-7660.jpg',\
        'D201711541_2019-05-21 11_28_51-lv1-39189-27851-3979-5553.jpg',\
        'D201802733_2019-05-14 15_41_01-lv1-18828-14936-7756-5310.jpg']
    
    image_path_list=[IMAGE_WRITE_PATH+'_train',IMAGE_WRITE_PATH+'_val',IMAGE_WRITE_PATH+'_test']
    for write_path in image_path_list:
        file_name=write_path.split('/')[-1]
        if not os.path.exists(write_path):
            os.mkdir(write_path)
            print('{} directory made'.format(file_name))
        else:
            print('{} directory exists'.format(file_name))
        
   
    mask_path_list=[MASK_WRITE_PATH+'_train',MASK_WRITE_PATH+'_val',MASK_WRITE_PATH+'_test']

    for write_path in mask_path_list:
        file_name=write_path.split('/')[-1]
        if not os.path.exists(write_path):
            os.mkdir(write_path)
            print('{} directory made'.format(file_name))
        else:
            print('{} directory exists'.format(file_name))

    if not os.path.exists(DISCARD_WRITE_PATH):
        os.mkdir(DISCARD_WRITE_PATH)
        print('discard directory made')
    else:
        print('discard directory exists')

    image_list=os.listdir(IMAGE_PATH)
    for x in must_include:
        image_list.remove(x)
    
    
    random.shuffle(image_list)
    train_list_temp=image_list[:int(0.80*len(image_list))]

    train_list=train_list_temp[:int(0.80*len(train_list_temp))]
    train_list.extend(must_include)
    validation_list=train_list_temp[int(0.80*len(train_list_temp)):]
    test_list=image_list[int(0.80*len(image_list)):]
    print('TRAIN PATCHES')
    gen_patches(train_list,IMAGE_WRITE_PATH+'_train',MASK_WRITE_PATH+'_train',DISCARD_WRITE_PATH)
    print('VALIDATION PATCHES')
    gen_patches(validation_list,IMAGE_WRITE_PATH+'_val',MASK_WRITE_PATH+'_val',DISCARD_WRITE_PATH)
    print('TEST PATCHES')
    gen_patches(test_list,IMAGE_WRITE_PATH+'_test',MASK_WRITE_PATH+'_test',DISCARD_WRITE_PATH)


                    
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH_NOR='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
IMAGE_PATH_NN='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
IMAGE_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_overlap'
MASK_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_overlap'
DISCARD_WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/discard_overlap'


#extract_overlap_patches(MASK_PATH,IMAGE_PATH_NN,IMAGE_WRITE_PATH,MASK_WRITE_PATH,DISCARD_WRITE_PATH)


In [ ]:
def create_masked_imgs():
    MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
    IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
    WRITE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/masked_img'
    if not os.path.exists(WRITE_PATH):
        os.mkdir(WRITE_PATH)
        print('{} directory made'.format(WRITE_PATH))
    else:
        print('{} directory exists'.format(WRITE_PATH))


    image_list=os.listdir(IMAGE_PATH)

    for image_ID in tqdm(image_list):
        # Format of image_ID : D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942.jpg
        image_name=os.path.join(IMAGE_PATH,image_ID)
        bgr_img=cv2.imread(image_name,1)
        mask_ID=image_ID.split('.')[0]+'_mask.'+image_ID.split('.')[-1]
        # Format of mask_ID: D20180056701_2019-05-21 09_50_58-lv1-27237-28645-8099-3942_mask.jpg
        mask_name=os.path.join(MASK_PATH,mask_ID)
        mask_img=cv2.imread(mask_name,0)
        ret,mask_img=cv2.threshold(mask_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)


        temp_x,temp_y=np.where(mask_img==0)
        bgr_img[temp_x,temp_y,:]=0
        cv2.imwrite(os.path.join(WRITE_PATH,image_ID),bgr_img)
#create_masked_imgs()



In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self,image_IDs,IMAGE_PATH,MASK_PATH,batch_size=32,sample_wise=False,feature_wise=False,scale=1,\
                 shuffle=True):
        print("Number of sample : {}".format(len(image_IDs)))

        self.image_IDs = image_IDs
        self.IMAGE_PATH=IMAGE_PATH
        self.MASK_PATH=MASK_PATH
        self.batch_size=batch_size
        self.sample_wise=sample_wise
        self.feature_wise=feature_wise
        self.scale=scale
        self.shuffle = shuffle
        if self.feature_wise==True:
            self.mean_value=np.zeros(3)#np.array([154.55561068,100.37918644,167.97885898])
            self.std_value=np.ones(3)#np.array([50.64775462,55.05548022,36.14331006])
        self.on_epoch_end()
        

    def __len__(self):
        return int(np.ceil(len(self.image_IDs) / self.batch_size))
    

    def __getitem__(self, index):
    
        image_ID_temp= self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X=[]
        Y=[]
        for ID in image_ID_temp:
            x, y = self.__data_generation(ID)
            X.append(x)
            Y.append(y)
            
        X=np.array(X)
        Y=np.array(Y)

        return X, Y

    def on_epoch_end(self):
        
        self.indices = np.arange(len(self.image_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            

    def __data_generation(self, image_ID_temp):
        
        img_path=os.path.join(self.IMAGE_PATH,self.image_IDs[image_ID_temp])
        
        mask_name=self.image_IDs[image_ID_temp].split('.')[0]+'_mask.jpg'
        mask_path=os.path.join(self.MASK_PATH,mask_name)
        
        X = imread(img_path)
        #X=resize(X,(1024,1024,3))
        
        if self.scale!=1 and self.sample_wise==True:
            sys.exit("Scale and sample_wise, both cannot be implemneted")
        
        X=X*self.scale
        
        if self.sample_wise==True:
            X=np.array(X,np.float64)
            X[:,:,0]=(X[:,:,0]-np.mean(X[:,:,0]))/np.std(X[:,:,0])
            X[:,:,1]=(X[:,:,1]-np.mean(X[:,:,1]))/np.std(X[:,:,1])
            X[:,:,2]=(X[:,:,2]-np.mean(X[:,:,2]))/np.std(X[:,:,2])
            
        if self.feature_wise==True:
            X=np.array(X,np.float64)
            X=(X-self.mean_value)/self.std_value
        #X=np.expand_dims(X,axis=0)
        y=imread(mask_path,as_gray=True)
        y=y/255
        y=np.expand_dims(y,axis=2)


        return X,y

In [ ]:
def train_test_array(): 
    x=[]
    y=[]
    INPUT_IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_non_normalized'
    OUTPUT_MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_non_normalized'
    sample_wise=True
    imge_name=os.listdir(INPUT_IMAGE_PATH)
    for name in tqdm(imge_name):
        img=imread(os.path.join(INPUT_IMAGE_PATH,name))
        img=np.array(img,np.float64)
        if sample_wise==True:
                img[:,:,0]=(img[:,:,0]-np.mean(img[:,:,0]))/np.std(img[:,:,0])
                img[:,:,1]=(img[:,:,1]-np.mean(img[:,:,1]))/np.std(img[:,:,1])
                img[:,:,2]=(img[:,:,2]-np.mean(img[:,:,2]))/np.std(img[:,:,2])
        mask_name=name.split('.')[0]+'_mask.jpg'
        mask_path=os.path.join(OUTPUT_MASK_PATH,mask_name)
        mask=imread(mask_path)
        mask=np.array(mask)
        mask=np.expand_dims(mask,axis=2)
        x.append(img)
        y.append(mask/255)

    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,shuffle=True,random_state=42)  
    return train_x,test_x,train_y,test_y

#train_x,test_x,train_y,test_y=train_test_array()


In [ ]:
def get_mean_std(train_list):  
    IMAGE_PATH_NOR='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1'
    INPUT_IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_cn_overlap_train'
    
    x_sum=np.zeros((512,512,3))
    for img in tqdm(train_list):
        path=os.path.join(INPUT_IMAGE_PATH,img)
        x_sum=x_sum+imread(path)
    img_sum=np.sum(x_sum,axis=(0,1))
    img_mean=img_sum/(len(train_list)*(512*512))    
    x_var=np.zeros((512,512,3))
    for img in tqdm(train_list):
        path=os.path.join(INPUT_IMAGE_PATH,img)
        x_var+=np.square(imread(path)-img_mean)
    img_var=np.sum(x_var,axis=(0,1))
    img_var=img_var/(len(train_list)*(512*512))
    img_std=np.sqrt(img_var)
    
    
    #img_mean=np.array([172.74560205,110.46180509,175.13698655])
    #img_std=np.array([41.87534665,51.27207546,35.44355781])
    print(img_mean,img_std)
    return img_mean,img_std

In [ ]:
!ls

In [ ]:
INPUT_IMAGE_PATH_TRAIN='input_cn_overlap_train'
INPUT_IMAGE_PATH_VAL='input_cn_overlap_val'

OUTPUT_MASK_PATH_TRAIN='mask_cn_overlap_train'
OUTPUT_MASK_PATH_VAL='mask_cn_overlap_val'

train_list=os.listdir(INPUT_IMAGE_PATH_TRAIN)

val_list=os.listdir(INPUT_IMAGE_PATH_VAL)
train_generator=DataGenerator(train_list,INPUT_IMAGE_PATH_TRAIN,OUTPUT_MASK_PATH_TRAIN,batch_size=8\
                              ,feature_wise=False,scale=1/255)
val_generator=DataGenerator(val_list,INPUT_IMAGE_PATH_VAL,OUTPUT_MASK_PATH_VAL,batch_size=8,\
                            feature_wise=False,scale=1/255)
hard_code_val=False


if hard_code_val:
    train_generator.mean_value=[172.4971903,110.05776942,174.9252121] 
    train_generator.std_value=[41.74648248,50.71953652,35.17684964]

    val_generator.mean_value=[172.4971903,110.05776942,174.9252121] 
    val_generator.std_value=[41.74648248,50.71953652,35.17684964]



In [ ]:
if train_generator.feature_wise and not hard_code_val:
    print('Changing mean and std')
    val_generator.feature_wise=True
    mean,std=get_mean_std(train_list)
    train_generator.mean_value=mean
    train_generator.std_value=std
    val_generator.mean_value=mean
    val_generator.std_value=std
else:
    print("Not Changing mean and std")
    #print(train_generator.mean_value,train_generator.std_value)
    #print(val_generator.mean_value,val_generator.std_value)

mean and std for overlap [172.74560205,110.46180509,175.13698655] [41.87534665,51.27207546,35.44355781]

In [ ]:
def visualize_data(index): 
    g_x,g_y=train_generator.__getitem__(index)
    for i in range(g_x.shape[0]):
        print(g_x[i].shape)
    print("Shape of batches of input: {} \nShape of batches of output: {}".format(g_y.shape,g_x.shape))
    g_x=g_x[0]
    g_y=g_y[0]
    print(np.amax(g_y),np.amax(g_x))
    print(train_generator.feature_wise)
    if train_generator.feature_wise==True:
        mean_value=train_generator.mean_value
        std_value=train_generator.std_value
    else:
        mean_value=np.zeros(3)
        std_value=np.ones(3)
    scale=train_generator.scale
        
    g_x=(g_x*std_value+mean_value)/scale
    g_x=g_x.astype(np.uint8)
    
    fig=plt.figure()
    plt.imshow(g_x)
    fig2=plt.figure()
    plt.imshow(np.squeeze(g_y,axis=2),cmap=plt.cm.gray)
    print(np.amax(g_y),np.amax(g_x))
visualize_data(15)

In [ ]:
class CosineWithRestart(keras.callbacks.Callback):
    
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=3,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)
        
        
class CheckLR(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs):
        print("LR : {} for epoch {}".format(K.eval(self.model.optimizer.lr),epoch))
        
def dice_coef(y_true, y_pred):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.round(K.flatten(y_pred))
    intersection = K.sum(y_true_f * y_pred_f)
    union=K.sum(y_true_f) + K.sum(y_pred_f)
    
    return (2. * intersection) / ( union+ K.epsilon())


def recall(y_true, y_pred):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)
    fn=K.sum((1-y_pred_f)*y_true_f)
    recall= tp/(tp+fn+K.epsilon())
    return recall

def auc(y_true, y_pred):
    #y_true_f = K.flatten(y_true)
    #y_pred_f = K.flatten(y_pred)
    auc = tf.metrics.auc(y_true, y_pred,curve='PR',summation_method="careful_interpolation")[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

def cm (y_true, y_pred):
    
    auc = tf.metrics.auc(y_true, y_pred,curve='PR',summation_method="careful_interpolation")[1]
    K.get_session().run(tf.local_variables_initializer())
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    union=K.sum(y_true_f) + K.sum(y_pred_f)
    
    dice= (2. * intersection) / ( union+ K.epsilon())
    
    return (dice*100+auc*100)/2
    




In [ ]:


def unet(pretrained_weights = None,input_size = (None,None,3),act='relu',optimizer='sgd'):
    inputs = Input(input_size)
    assert act in ['relu','leaky_relu']
    assert optimizer in ['adam','sgd']
    if act=='relu':
        activation1='relu'
    else:
        activation1=LeakyReLU(alpha=0.5)
    activation='relu'
    
    
    
    
    conv1 = Conv2D(64, 3, activation = activation1, padding = 'same', kernel_initializer = 'he_normal')(inputs)
    #bach_size x 512 x 512 x 64
    conv1 = Conv2D(64, 3, activation = activation1, padding = 'same', kernel_initializer = 'he_normal')(conv1)
    #bach_size x 512 x 512 x 64
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #bach_size x 256 x 256 x 64
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool1)
    #bach_size x 256 x 256 x 128
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    #bach_size x 256 x 256 x 128
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #bach_size x 128 x 128 x 128
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool2)
    #bach_size x 128 x 128 x 256
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv3)
    #bach_size x 128 x 128 x 256
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #bach_size x 64 x 64 x 256
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool3)
    #bach_size x 64 x 64 x 512
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv4)
    #bach_size x 64 x 64 x 512
    drop4 = Dropout(0.5)(conv4)
    
    #pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    #drop5 = Dropout(0.5)(conv5)

    #up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #merge6 = concatenate([drop4,up6], axis = 3)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    
    
    up7 = Conv2D(256, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))#(conv6))
    #bach_size x 128 x 128 x 256
    merge7 = concatenate([conv3,up7], axis = 3)
    #bach_size x 128 x 128 x 512
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge7)
    #bach_size x 128 x 128 x 256
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #bach_size x 128 x 128 x 256

    up8 = Conv2D(128, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #bach_size x 256 x 256 x 128
    merge8 = concatenate([conv2,up8], axis = 3)
    #bach_size x 256 x 256 x 256
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge8)
    #bach_size x 256 x 256 x 128
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv8)
    #bach_size x 256 x 256 x 128

    up9 = Conv2D(64, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #bach_size x 512 x 512 x 64
    merge9 = concatenate([conv1,up9], axis = 3)
     #bach_size x 512 x 512 x 128
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(2, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 2
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
     #bach_size x 512 x 512 x 1

    model = Model(input = inputs, output = conv10)
    
    if optimizer=='adam':
        OPT=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0000, amsgrad=False)
        print('adam optimizer')
    elif optimizer=='sgd':
        OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
        print('SGD optimizer')
    #dice_coef_loss
    model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,recall,auc])    

    return model



In [ ]:
def expend_as(tensor, rep):
        my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep})(tensor)
        return my_repeat

def AttnGatingBlock(x, g, inter_shape):
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same')(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same')(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    # my_repeat=Lambda(lambda xinput:K.repeat_elements(xinput[0],shape_x[1],axis=1))
    # upsample_psi=my_repeat([upsample_psi])
    upsample_psi = expend_as(upsample_psi, shape_x[3])

    y = multiply([upsample_psi, x])

    # print(K.is_keras_tensor(upsample_psi))

    result = Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = BatchNormalization()(result)
    return result_bn

def UnetGatingSignal(input, is_batchnorm=True):
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 2, (1, 1), strides=(1, 1), padding="same")(input)
    if is_batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def attn_unet(pretrained_weights = None,input_size = (512,512,3),act='relu',optimizer='adam'):
    inputs = Input(input_size)
    assert act in ['relu','leaky_relu']
    assert optimizer in ['adam','sgd']
    if act=='relu':
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        #bach_size x 512 x 512 x 64
        conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        #bach_size x 512 x 512 x 64
    else:
        conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1=LeakyReLU(alpha=0.5)(conv1)
        #bach_size x 512 x 512 x 64
        conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv1=LeakyReLU(alpha=0.5)(conv1)
        #bach_size x 512 x 512 x 64
    activation='relu'
  
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #bach_size x 256 x 256 x 64
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool1)
    #bach_size x 256 x 256 x 128
    conv2 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv2)
    #bach_size x 256 x 256 x 128
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #bach_size x 128 x 128 x 128
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool2)
    #bach_size x 128 x 128 x 256
    conv3 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv3)
    #bach_size x 128 x 128 x 256
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #bach_size x 64 x 64 x 256
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(pool3)
    #bach_size x 64 x 64 x 512
    conv4 = Conv2D(512, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv4)
    #bach_size x 64 x 64 x 512
    drop4 = Dropout(0.5)(conv4)
    
    #pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    #conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    #drop5 = Dropout(0.5)(conv5)

    #up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #merge6 = concatenate([drop4,up6], axis = 3)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    #conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    
    
    g1=UnetGatingSignal(drop4)
    #bach_size x 64 x 64 x 1024
    att1=AttnGatingBlock(conv3, g1, 512)
    #bach_size x 128 x 128 x 256
    up7 = Conv2D(256, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))#(conv6))
    #bach_size x 128 x 128 x 256
    merge7 = concatenate([att1,up7], axis = 3)
    #bach_size x 128 x 128 x 512
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge7)
    #bach_size x 128 x 128 x 256
    conv7 = Conv2D(256, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #bach_size x 128 x 128 x 256
    
    g2=UnetGatingSignal(conv7)
    #bach_size x 128 x 128 x 512
    att2=AttnGatingBlock(conv2, g2, 256)
    #bach_size x 256 x 256 x 128

    up8 = Conv2D(128, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #bach_size x 256 x 256 x 128
    merge8 = concatenate([att2,up8], axis = 3)
    #bach_size x 256 x 256 x 256
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge8)
    #bach_size x 256 x 256 x 128
    conv8 = Conv2D(128, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv8)
    #bach_size x 256 x 256 x 128
    
    g3=UnetGatingSignal(conv8)
    #bach_size x 256 x 256 x 256
    att3=AttnGatingBlock(conv1, g3, 128)
    #bach_size x 512 x 512 x 64

    up9 = Conv2D(64, 2, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #bach_size x 512 x 512 x 64
    merge9 = concatenate([att3,up9], axis = 3)
     #bach_size x 512 x 512 x 128
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(merge9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 64
    conv9 = Conv2D(2, 3, activation = activation, padding = 'same', kernel_initializer = 'he_normal')(conv9)
     #bach_size x 512 x 512 x 2
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
     #bach_size x 512 x 512 x 1

    model = Model(inputs = inputs, outputs = conv10)
    
    if optimizer=='adam':
        OPT=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0000, amsgrad=False)
        print('adam optimizer')
    elif optimizer=='sgd':
        OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
        print('SGD optimizer')
    model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef])
    

    return model



In [ ]:
model=attn_unet(input_size = (512,512,3),act='relu',optimizer='sgd')
model.summary()

In [ ]:
TRAIN_STEPS=train_generator.__len__()
VAL_STEPS=val_generator.__len__()

model_start_date=datetime.datetime.now().strftime("%Y_%m_%d")
dir_name = os.path.join(os.getcwd(),"model_non_overlap_unet{}".format(model_start_date))

if os.path.exists(dir_name):
    print("model already exists in {}".format("model_non_overlap_unet{}".format(model_start_date)))
else:
    os.mkdir(dir_name)
    print(" dir {} made".format("model_non_overlap_unet{}".format(model_start_date)))

In [ ]:
reduce_lr_on_plateau_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,mode='min',min_delta=0.001,\
                     patience=4,cooldown=0,min_lr= 1e-12,verbose=1)
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 2
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_sceduler=keras.callbacks.LearningRateScheduler(step_decay, verbose=1)

model_checkpoint_cb=keras.callbacks.ModelCheckpoint('{}/model_optimum_{}.hdf5'.\
                                      format(dir_name,datetime.datetime.now().strftime("%Y_%m_%d")),\
                                      monitor='val_dice_coef', verbose=1, save_best_only=True, \
                                      save_weights_only=True, mode='max', period=1)


cosine_annealing_cb=CosineWithRestart(1e-06,0.01,TRAIN_STEPS,lr_decay=1,cycle_length=8,mult_factor=1.5)
print_lr_cb=CheckLR()
callbacks=[model_checkpoint_cb,print_lr_cb,cosine_annealing_cb]

In [ ]:

history=model.fit_generator(generator=train_generator,epochs=38,verbose=1,\
                            callbacks=callbacks,\
                                 validation_data=val_generator,use_multiprocessing=True)
#model.fit(train_x, train_y, epochs=600, batch_size=1, verbose=1, shuffle=False,validation_data=(test_x,test_y))

fig=plt.figure()
plt.plot(history.history['loss'],label='training_loss')
plt.plot(history.history['val_loss'],label='validation_loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('loss')
fig.savefig('{}/plot_loss_{}.PNG'.format(dir_name,model_start_date))

f1=history.history['dice_coef']
f1_test=history.history['val_dice_coef']

fig2=plt.figure()
plt.plot(f1,label='training')
plt.plot(f1_test,label='testing')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Dice_score')
fig2.savefig('{}/plot_dice_coef_{}.PNG'.format(dir_name,model_start_date))

fig3=plt.figure()
plt.plot(history.history['recall'],label='training')
plt.plot(history.history['val_recall'],label='testing')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('recall')
fig3.savefig('{}/plot_recall_{}.PNG'.format(dir_name,model_start_date))

fig4=plt.figure()
plt.plot(history.history['auc'],label='training')
plt.plot(history.history['val_auc'],label='testing')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('AUC')
fig4.savefig('{}/plot_auc_{}.PNG'.format(dir_name,model_start_date))

print("Training dice score: {}\nValidation dice score: {}".format(f1[-1],f1_test[-1]))

model_json = model.to_json()
with open("{}/model_{}.json".format(dir_name,model_start_date), "w") as json_file:
    json_file.write(model_json)
model.save_weights("{}/model_final_weights_{}.h5".format(dir_name,model_start_date))
print("Saved model to disk") 


In [ ]:
INPUT_IMAGE_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_overlap_test/'
OUTPUT_MASK_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/mask_overlap_test/'

test_list=os.listdir(INPUT_IMAGE_PATH_TEST)
test_generator=DataGenerator(test_list,INPUT_IMAGE_PATH_TEST,OUTPUT_MASK_PATH_TEST,\
                            batch_size=8,feature_wise=False,scale=1/255)

# LOADING MODEL

In [ ]:
dir_content=os.listdir()
model_list=[s for s in dir_content if 'model_non_cn' in s]
model_list=sorted(model_list)
model_folder=os.path.join(os.getcwd(),model_list[-1])
print("available models are :",model_list,"\n","selected model is in :", model_folder)
model_weights=os.path.join(os.getcwd(),model_folder)+"/"+[weights for weights in os.listdir(model_folder) \
                                                          if '.hdf5' in weights][-1]
model_architecture=glob.glob(os.path.join(os.getcwd(),model_folder)+"/*.json")[-1]
print("Selected model file is : {}\nSelected architecture file is : {}".format(model_weights.split('/')[-1]\
                                                                          ,model_architecture.split('/')[-1]))


In [ ]:
json_file = open(model_architecture, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
loaded_model.load_weights(model_weights)
loaded_model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,recall])
print(loaded_model.metrics_names)
loaded_model.summary()

In [ ]:
test_loss, test_acc, test_dice_coef, test_recall=loaded_model.evaluate_generator(test_generator,verbose=1,use_multiprocessing=True)
print("Test loss: {}\nTest accuracy : {}\nTest dice score : {}\nTest recall : {}"\
      .format(test_loss,test_acc,test_dice_coef,test_recall))

In [ ]:
from skimage import img_as_uint
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def combined_metric(y_true, y_pred):
    
    y_true_f = (y_true).reshape((-1,1))
    y_pred_f = np.round((y_pred).reshape((-1,1)))
    

    intersection = np.nansum(y_true_f * y_pred_f)+(np.finfo(float).eps)
    union=np.nansum(y_true_f) + np.nansum(y_pred_f)
    
    dice= (2 * intersection) / ( union+ (np.finfo(float).eps))
    
    
    
    return dice

def whole_img_pred(IMAGE_PATH,img_list,pred_dir_name,MASK_PATH=None,cal_performace=True):
    
    pred_dir=os.path.join(os.getcwd(),pred_dir_name)
    
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
        print("Made {} directory".format(pred_dir.split('/')[-1]))
    else:
        print("{} directory already exists in {}".format(pred_dir.split('/')[-1],'/'.join(pred_dir.split('/')[:-1])))
        
        

    avg_score=0
    test_num=len(img_list)
    for img_name in img_list:
        str_time=time.time()

        img=imread(os.path.join(IMAGE_PATH,img_name))
        r,c,_=img.shape#4663,3881
      
        new_r_count=(math.ceil((r-512)/512)+1)#10
        new_c_count=(math.ceil((c-512)/512)+1)#8


        pad_r1=((new_r_count-1)*512-r+512)//2 #228
        pad_r2=((new_r_count-1)*512-r+512)-pad_r1 #229
        pad_c1=((new_c_count-1)*512-c+512)//2 #107
        pad_c2=((new_c_count-1)*512-c+512)-pad_c1#108

        arr_in=np.pad(img, [(pad_r1,pad_r2),(pad_c1,pad_c2),(0,0)], 'constant', constant_values=0)/255
        window_shape=(512,512,3)
        arr_out=skimage.util.view_as_windows(arr_in, window_shape, step=512)
        x,y=arr_out.shape[:2]
        ar2=arr_out.reshape((-1,512,512,3))
        del arr_out,arr_in,img
           
        y_pred=loaded_model.predict(ar2)
        
    
        img_temp=[]
        for i in range(x):
            img_temp.append(np.concatenate(y_pred[i*y:(i+1)*y],axis=1))
        img_temp=np.array(img_temp,dtype=np.float_)

        img_temp=np.concatenate(img_temp,axis=0)
        img_temp=img_temp[pad_r1:img_temp.shape[0]-pad_r2,pad_c1:img_temp.shape[1]-pad_c2,:]*255
        thresh = threshold_otsu(img_temp)
        img_temp = np.array(img_temp > 0.8,dtype=np.float32)
        #img_temp=scipy.ndimage.morphology.binary_fill_holes(img_temp)
        pred_img_name=img_name.split('.')[0]+'_pred.jpg'
        pred_img_path=os.path.join(pred_dir,pred_img_name)
        skimage.io.imsave(pred_img_path,img_as_uint(img_temp))
        print("time for {} shaped {} is {} s".format((r,c),img_name,time.time()-str_time))
        if cal_performace:
            mask_path=os.path.join(MASK_PATH,img_name.split('.')[0]+'_mask.jpg')
            mask=imread(mask_path,as_gray=True)//255
            score=combined_metric(mask, img_temp)
            avg_score+=score
            print("Score : {}".format(score))
        del img_temp,ar2
        if cal_performace:
            del mask
    if cal_performace:
        print("Average Score : {}".format(avg_score/test_num))
    
    print("DONE")
        
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'

#NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg'
#neg_img_list=os.listdir(NEG_IMAGE_PATH)

all_img_list=os.listdir(IMAGE_PATH)

test_files=os.listdir(INPUT_IMAGE_PATH_TEST)
test_img_list=['_'.join(x.split('_')[:-2])+'.jpg' for x in test_files]


whole_img_pred(IMAGE_PATH,test_img_list,'whole_pred_vanillaUnet_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
               MASK_PATH,cal_performace=True)


        
        
        
        

In [ ]:
!nvidia-smi

In [ ]:
print(test_img_list)

In [ ]:
NEG_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg/'
images=[NEG_PATH+x for x in os.listdir(NEG_PATH)]
whole_img_pred(NEG_PATH,images,'whole_pred_neg')


In [ ]:
def visualize_performance(index,model):
    x,y=test_generator.__getitem__(index)
    y_pred=loaded_model.predict(x)
    temp=test_generator.indices[index*test_generator.batch_size:(index+1)*test_generator.batch_size]
    print(test_generator.image_IDs[temp[0]])
    y_pred_temp=np.squeeze(y_pred[0],axis=2)
    y_true_temp=np.squeeze(y[0],axis=2)
    fig=plt.figure()
    plt.imshow(y_true_temp,cmap=plt.cm.gray)
    fig1=plt.figure()
    plt.imshow(y_pred_temp,cmap=plt.cm.gray)
    fig2=plt.figure()
    plt.imshow(x[0])
    
    
visualize_performance(12,loaded_model)



In [ ]:

model2_folder=os.path.join(os.getcwd(),model_list[-3])

model2_weights=os.path.join(os.getcwd(),model_folder)+"/"+[weights for weights in os.listdir(model2_folder) \
                                                          if 'model_op' in weights][-1]
model2_architecture=glob.glob(os.path.join(os.getcwd(),model2_folder)+"/*.json")[-1]

json_file2 = open(model2_architecture, 'r')
loaded_model_json2 = json_file2.read()
json_file2.close()
loaded_model2 = model_from_json(loaded_model_json2)
OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
loaded_model2.load_weights(model_weights)
loaded_model2.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy',dice_coef,auc])
print("Selected model2 file is : {}\nSelected architecture2 file is : {}".format(model2_weights.split('/')[-1]\
                                                                          ,model2_architecture.split('/')[-1]))



In [ ]:
visualize_performance(12,loaded_model2)
